# **Preprocesamiento de los datos**
---
### Se fusionaran todos los archivos con extension .pcap del conjunto de datos CTU-13:

In [17]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data_path = './CTU-13_data'

data_file_names = [file for file in os.listdir(data_path)] # Lectura de los nombres de los archivos del directorio
#print(data_file_names)  # Lista de strings que contiene el nombre de los archivos .parquet

### Iteramos sobre los archivos que estan en la ruta especificada en ``` data_path = './CTU-13_data' ``` donde se hace la lectura de los archivos de .pcap para convertirlos a .csv.

In [ ]:
# Diccionario que contendra los dataframes despues de hacer el cast de parquet a pandas df
data_frames = {}
'''
Iteramos sobre la lista de nombres para añadirlos al diccionario de dataframes

Ejemplo del formato del diccionario seria: 

    {'1-Neris-20110810.binetflow.parquet': pd.DataFrame('./CTU-13_data/1-Neris-20110810.binetflow.parquet')}

'''
for file in data_file_names:
    data_frames[file] = pd.read_parquet(f'{data_path}/{file}') 

### Ahora fusionamos todos los datasets

In [ ]:
all_data = pd.concat([ data_frames[file] for file in data_file_names ])
print(all_data.head())
print(all_data.shape)

        dur proto    dir state  stos  dtos  tot_pkts  tot_bytes  src_bytes  \
0  1.026539   tcp     ->  S_RA   0.0   0.0         4        276        156   
1  1.009595   tcp     ->  S_RA   0.0   0.0         4        276        156   
2  3.056586   tcp     ->  SR_A   0.0   0.0         3        182        122   
3  3.111769   tcp     ->  SR_A   0.0   0.0         3        182        122   
4  3.083411   tcp     ->  SR_A   0.0   0.0         3        182        122   

                                    label                  Family  
0  flow=Background-Established-cmpgw-CVUT  20110810.binetflow.csv  
1  flow=Background-Established-cmpgw-CVUT  20110810.binetflow.csv  
2             flow=Background-TCP-Attempt  20110810.binetflow.csv  
3             flow=Background-TCP-Attempt  20110810.binetflow.csv  
4             flow=Background-TCP-Attempt  20110810.binetflow.csv  
(10598771, 11)
